# Information

Creating a binary text classification model to check if a string of text to the text-to-SQL model or another model

# Install libraries

In [1]:
!pip install transformers datasets scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Import libraries

In [2]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import shutil
from google.colab import files
import os

# Generate dataset


In [7]:
import random
import pandas as pd

# Define SQL and Non-SQL templates
sql_templates = [
    "Can you tell me how many {category} I bought {period}?",
    "I'd like to see the total {category} for {period}, please.",
    "How many {category} did we receive {period}? I need that info for a report.",
    "Could you show me the most popular {category} last year?",
    "Can you find out the average {metric} of {category} in {department}?",
    "Tell me the number of {category} who placed an order in {period}.",
    "I want to know how much {metric} we earned last {period}.",
    "Can you tell me who the top {number} {category} are this year?",
    "How many {category} signed up in the last {period}?",
    "Show me the details for all {category} from {customer}.",
    "Can you tell me how many {category} we sold {period}?",
    "What were the total {category} for {period}?",
    "Show me all {category} that were sold in {period}.",
    "How many {category} did we receive {period}?",
    "List the {category} that were sold the most in {period}.",
    "What’s the average {metric} of {category} in {department}?",
    "Give me the details of the {category} made by {customer}.",
    "Who are the top {number} {category} in {period}?",
    "Show me all {category} placed in {period}.",
    "How many {category} signed up in {period}?",
    "Can you tell me how many {category} I bought {period}?",
    "I'd like to see the total {category} for {period}, please.",
    "How many {category} did we receive {period}? I need that info for a report.",
    "Could you show me the most popular {category} last year?",
    "Find a user with the name {name}.",
    "Retrieve details for someone called {name}.",
    "Do we have any records for a user named {name}?",
    "Look up the information for a user called {name}.",
    "Fetch the data for an account belonging to {name}.",
    "Tell me the number of {category} who placed an order in {period}.",
]

non_sql_templates = [
    "What’s the weather like {location}?",
    "Can you remind me to {task} later?",
    "What’s the time of my meeting with {person}?",
    "Where is the best place to eat {food} in {city}?",
    "What’s the capital of {country}?",
    "Can you tell me how to get to the nearest {place}?",
    "When is the next {event}?",
    "Who won the {award} in {period}?",
    "How tall is {place}?",
    "What are the ingredients for {recipe}?",
    "Remind me to call my friend later.",
    "Can you remind me to go for a walk in 30 minutes?",
    "Please set a reminder to email my boss.",
    "When is my dentist appointment?",
    "What time is my flight tomorrow?",
    "Can you tell me what time my meeting with {person} starts?",
    "Remind me to take my medicine at 6 PM.",
    "What’s the weather like tomorrow?",
    "Can you send me a reminder for my anniversary next week?",
    "Please remind me to pick up my dry cleaning after work.",
    "What’s the weather like {location}?",
    "Can you remind me to {task} later?",
    "Where is the best place to eat {food} in {city}?",
    "What’s the capital of {country}?",
    "Can you tell me how to get to the nearest {place}?",
    "When is the next {event}?",
    "Who won the {award} in {period}?",
    "How tall is {place}?",
    "What are the ingredients for {recipe}?",
    "Please remind me to pick up my dry cleaning.",
]

# Define possible variations
categories = ["products", "orders", "sales", "customers", "users"]
periods = ["last month", "this week", "last quarter", "2023", "this year"]
metrics = ["price", "age", "revenue"]
departments = ["IT", "sales", "marketing"]
names = ["Ashp116", "JohnDoe", "Jane123", "Bob47", "Alice99"]
locations = ["Tokyo", "Paris", "New York", "London"]
tasks = ["buy milk", "pick up groceries", "call my mom"]
food = ["sushi", "pizza", "burger"]
cities = ["Tokyo", "LA", "Paris", "London"]
places = ["Mount Everest", "the Eiffel Tower", "the Great Wall of China"]
events = ["SpaceX launch", "Super Bowl", "Oscar ceremony"]
award = ["Oscar", "Grammy", "Nobel Prize"]
recipe = ["chocolate cake", "pasta", "pizza"]
person = ["Sarah", "Bob", "John"]

# Function to generate dataset
def generate_data(num_samples):
    data = []
    for _ in range(num_samples // 2):  # For SQL queries (label 1)
        template = random.choice(sql_templates)
        sentence = template.format(
            category=random.choice(categories),
            period=random.choice(periods),
            metric=random.choice(metrics),
            department=random.choice(departments),
            name=random.choice(names),
            customer=random.choice(["customer X", "customer Y"]),
            number=random.randint(1, 10)
        )
        data.append([sentence, 1])

    for _ in range(num_samples // 2):  # For non-SQL queries (label 0)
        template = random.choice(non_sql_templates)
        sentence = template.format(
            location=random.choice(locations),
            task=random.choice(tasks),
            person=random.choice(person),
            category=random.choice(categories),
            period=random.choice(periods),
            food=random.choice(food),
            city=random.choice(cities),
            place=random.choice(places),
            event=random.choice(events),
            award=random.choice(award),
            recipe=random.choice(recipe),
            country="France"  # Fixed country example
        )
        data.append([sentence, 0])

    return pd.DataFrame(data, columns=["text", "label"])

# Generate two datasets: one for training and one for testing
train_df = generate_data(50000)  # 50,000 examples for training
test_df = generate_data(10000)  # 10,000 examples for testing

# Save the datasets to CSV files
train_path = "large_text_to_sql_train.csv"
test_path = "large_text_to_sql_test.csv"

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"Training dataset saved to {train_path}")
print(f"Testing dataset saved to {test_path}")

Training dataset saved to large_text_to_sql_train.csv
Testing dataset saved to large_text_to_sql_test.csv


# Create dataset

Creating a training and testing dataset from the files.

In [8]:
# Step 1: Load the datasets independently
train_data1 = load_dataset("csv", data_files=train_path)["train"]
test_data1 = load_dataset("csv", data_files=test_path)["train"]  # Adjusted to "train" for test data

# Concatenate the datasets (train + train and test + test)
combined_train_data = concatenate_datasets([train_data1])
combined_test_data = concatenate_datasets([test_data1])

# Combine both train and test datasets into one dataset
datasets_dict = {
    "train": combined_train_data,
    "test": combined_test_data
}

# Verify the combined dataset
print(f"Combined Dataset: {datasets_dict}")
print(f"Number of examples in combined dataset: {len(datasets_dict)}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Combined Dataset: {'train': Dataset({
    features: ['text', 'label'],
    num_rows: 50000
}), 'test': Dataset({
    features: ['text', 'label'],
    num_rows: 10000
})}
Number of examples in combined dataset: 2


# Tokenizer

 Setup tokenzier and tokenzier the datasets


In [9]:
# Step 2: Load tokenizer and model
model_name = "distilbert-base-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Step 3: Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_datasets = {
    "train": datasets_dict["train"].map(preprocess_function, batched=True),
    "test": datasets_dict["test"].map(preprocess_function, batched=True)
}

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Training

Traning the model

In [11]:
# Step 4: Define evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [12]:
# Step 5: Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
)

# Step 6: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Step 7: Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-adf708b34f21>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.000000,0.000003,1.000000,1.000000,1.000000,1.000000
2,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
3,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=9375, training_loss=0.0014102205989525343, metrics={'train_runtime': 235.2844, 'train_samples_per_second': 637.526, 'train_steps_per_second': 39.845, 'total_flos': 737367355800000.0, 'train_loss': 0.0014102205989525343, 'epoch': 3.0})

In [13]:
# Save the model and tokenizer
model.save_pretrained("./sql_classifier_large")
tokenizer.save_pretrained("./sql_classifier_large")

('./sql_classifier_large/tokenizer_config.json',
 './sql_classifier_large/special_tokens_map.json',
 './sql_classifier_large/vocab.txt',
 './sql_classifier_large/added_tokens.json',
 './sql_classifier_large/tokenizer.json')

# Export

Export the model

In [16]:
# Convert bytes to a more readable format (KB, MB, GB)
def convert_size(size_in_bytes):
    for unit in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size_in_bytes < 1024.0:
            return f"{size_in_bytes:.2f} {unit}"
        size_in_bytes /= 1024.0


# Step 1: Define the paths
output_dir = "/content/trained_model"  # The directory where the model is saved
zip_file_path = "/content/trained_model.zip"  # The location for the zip file

# Save the model
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


# Step 2: Zip the trained model directory
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', output_dir)

# Step 3: Check if the zip file exists and get the file size
if os.path.exists(zip_file_path):
    file_size = os.path.getsize(zip_file_path)
    print(f"File size: {file_size} bytes ({convert_size(file_size)})")
else:
    print("Failed to create the zip file.")

# Step 4: Trigger download of the zip file
files.download(zip_file_path)

Model and tokenizer saved to /content/trained_model
File size: 247311616 bytes (235.85 MB)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing

Testing the model

In [14]:
# Step 8: Inference function
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits.detach().cpu().numpy()
    prediction = np.argmax(logits, axis=1)[0]
    return "SQL" if prediction == 1 else "Other"

In [15]:
# Example usage
test_texts = [
    "List all customers who ordered more than $100.",
    "Send an email to Alice.",
    "Users",
    "list users",
    "list me all the users with the name ashp116",
    "I am looking for a user with the name Ashp116"
]
for text in test_texts:
    print(f"Input: {text} -> Prediction: {predict(text)}")

Input: List all customers who ordered more than $100. -> Prediction: SQL
Input: Send an email to Alice. -> Prediction: SQL
Input: Users -> Prediction: SQL
Input: list users -> Prediction: SQL
Input: list me all the users with the name ashp116 -> Prediction: SQL
Input: I am looking for a user with the name Ashp116 -> Prediction: SQL
